In [45]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime

1) Log file is loaded

In [46]:
file = open("log.txt", "r") 
lines = file.readlines()
for l in range(3): print(lines[l][:-1])

156.15.13.173 [01/Jan/2018:11:13:49 +0100] "GET AddProduct.html HTTP/1.0" 200 2356
176.247.135.49 [01/Jan/2018:11:18:07 +0100] "GET Shop.html HTTP/1.0" 200 7466
176.247.135.49 [01/Jan/2018:11:19:05 +0100] "GET AddProduct.html HTTP/1.0" 200 2356


2) Cass "Request" parses and keeps data of a single requests. Complete the TODO, where a given "line" (string) should be parsed in order to dervie the ip of the host making the request, time of the request (in seconds), and target page (name).

In [47]:
class Request:
    def __init__(self, line):
        # TODO DO PARSING HERE
        lista = line.split()
        self.ip = lista[0]
        date = lista[1]
        self.time = int(date[-2:]) + int(date[-5:-3]) * 60 + int(date[-8:-6]) * 60 * 60
        self.page = lista[4]
        # convert hours + minutes + seconds -> total number of seconds
        
    def __repr__(self):
        return self.ip + "     " +  str(datetime.timedelta(seconds=self.time)) + " -> " + self.page


3) Each line of log.txt is parsed and the Request objects are added to the list

In [48]:
reqs = [Request(line) for line in lines]
for l in range(10): print(reqs[l]) 

156.15.13.173     11:13:49 -> AddProduct.html
176.247.135.49     11:18:07 -> Shop.html
176.247.135.49     11:19:05 -> AddProduct.html
176.247.135.49     11:29:05 -> Summary.html
69.84.30.199     11:43:59 -> Shop.html
69.84.30.199     11:45:16 -> AddProduct.html
57.116.45.120     11:46:45 -> AddProduct.html
69.84.30.199     11:50:36 -> Summary.html
69.84.30.199     11:52:38 -> Payment.html
26.19.183.202     11:53:11 -> AddProduct.html


# Exercise

Your task is to analyze the log file. You should indentify the users and sessions. Then, compute the following: <br>
3.1) the number of sessions (should be 1000 :) ), <br>
3.2) the number of users, <br>
3.3) the average number of sessions per one user <br>
3.4) use the distribution plot to visualize the sessions' lengths (minutes);  <br>
3.5) use the distribution plot to visualize the sessions' lengths (the number of requests) <br>
3.6) use the distribution plot to visualize sessions' starting times (hours). <br>
3.7) which web page is the most common entry point?; for each page plot the number of times it has been an entry page,  <br>
3.8) which web page is the most common departure point?; for each page plot the number of times the user stopped a session while being on this page, <br>
3.9) for each page plot the average time the user stayed on this page <br>
3.10) print the fist 10 sessions (paths). <br>

You may find the following classes helpful and data.

In [49]:
class Session:
    def __init__(self, request):
        self.isClosed = False
        self.startTime = request.time
        self.stopTime = request.time
        self.requests = []
        self.requests.append(request)
        
    def __repr__(self):
        print("SESSION")
        print(self.startTime)
        print(self.stopTime)
        STR = ""
        for r in self.requests:
            print(r)
            STR += ("  " + r.page + "    " + str(datetime.timedelta(seconds=r.time)) + "\n")
        return STR

    def pushRequest(self, request):
        self.requests.append(request)
        self.stopTime = request.time

    def closeSession(self):
        self.isClosed = True

class Computer: # zmieniam rzeczy, jc to pacz original file
    def __init__(self, ip, session):
        self.ip = ip # [] # one user uses  only one computer (IP) to na chuj mi tablica ip????????
        self.sessions = []
        self.sessions.append(session)

    def __repr__(self):
        return self.ip

    def pushSession(self, session):
        self.sessions.append(session)

PAGES = ["Shop.html", "AddProduct.html", "Summary.html", "Payment.html", "About.html", "Contact.html"]

LINKS = [[1, 4], # tutaj sa polaczenia stron jak ktoras ma pusta liste to nastepny request z 
         [2,4], # innej strony to nowa sesja, a jak mozna isc dalej to ciag dalszy sesji
         [3],
         [],
         [1,5],
         []]

def isInTheSameSession(last, new): # przyjmuje requesty
    if last.page == new.page:
        return True
    elif len(LINKS[PAGES.index(last.page)]) == 0:
        return False
    elif LINKS.index(LINKS[PAGES.index(new.page)]) in LINKS[PAGES.index(last.page)]:
        return True

    return False

users = []
new_user = True # jesli taki user nie wystepuje w users i trzeba dodac

for req in reqs:
    sessionToAdd = None
    new_user = True
    for user in users:
        if req.ip != user.ip:
            continue
        
        new_user = False
        for session in user.sessions: # mysle ze rownie dobrze mozemy zamiast przechodzic po wszystkich sesja to od razu brac ostatnia
            if session.isClosed:
                continue
            elif isInTheSameSession(session.requests[-1], req): # tu powinno byc juz ostatnia sesja bo reszta zamknieta
                session.pushRequest(req)
            else: 
                session.closeSession()
                user.pushSession(Session(request=req))

        break

    if new_user:
        users.append(Computer(ip=req.ip, session=Session(req)))
        
        
#plot distribution
#count, bins, ignored = plt.hist(data, 50, alpha=0.5, normed=True)
#plt.plot()

#x_axis = np.arange(len(pages))
#plt.bar(x_axis, data, align='center', alpha=0.5)
#plt.xticks(y_pos, pages, rotation=90)